In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-17 18:52:37,428 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-17 18:52:37,429 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BH84" # rid dev: 2-BH84 prod: 2-C60W


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-17 18:52:37,745 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-17 18:52:37,746 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [6]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-17 18:52:41,185 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json] transfer successful. 0.60 KB transferred. Elapsed time: 0:00:00.000090.
2024-06-17 18:52:41,186 - INFO - Verifying SHA256 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json]
2024-06-17 18:52:41,202 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_feature_engineering',
    'rid': '2-BH7Y'}]},
 'execution_rid': '2-BH96',
 'workflow_rid': '2-BH80',
 'bag_paths': [PosixPath('/data/2-6162_78f365ad18323a68798b509668d599ba81e4740e90b130891750fa9a823179ac/Dataset_2-6162')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json')}

In [38]:
# Compute condition labels
icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])

condition_labels = EA.compute_condition_label(icd10_asso, icd10)

In [39]:
condition_labels

,Clinical_Records,Condition_Label
4532,2-6936,POAG
0,2-6938,POAG
4534,2-693A,POAG
2,2-693C,POAG
4536,2-693E,POAG
...,...,...
4523,2-6HHY,GS
9063,2-6HJ0,GS
4526,2-6HJ2,GS
9065,2-6HJ4,GS


In [41]:
# Insert new condition labels
EA.insert_condition_label(condition_labels)


[{'RID': '2-6936', 'Condition_Label': '2-BH7P'}, {'RID': '2-6938', 'Condition_Label': '2-BH7P'}, {'RID': '2-693A', 'Condition_Label': '2-BH7P'}, {'RID': '2-693C', 'Condition_Label': '2-BH7P'}, {'RID': '2-693E', 'Condition_Label': '2-BH7P'}, {'RID': '2-693G', 'Condition_Label': '2-BH7P'}, {'RID': '2-693J', 'Condition_Label': '2-BH7P'}, {'RID': '2-693M', 'Condition_Label': '2-BH7P'}, {'RID': '2-693P', 'Condition_Label': '2-BH7P'}, {'RID': '2-693R', 'Condition_Label': '2-BH7P'}, {'RID': '2-693T', 'Condition_Label': '2-BH7M'}, {'RID': '2-693W', 'Condition_Label': '2-BH7M'}, {'RID': '2-693Y', 'Condition_Label': '2-BH7M'}, {'RID': '2-6940', 'Condition_Label': '2-BH7M'}, {'RID': '2-6942', 'Condition_Label': '2-BH7M'}, {'RID': '2-6944', 'Condition_Label': '2-BH7M'}, {'RID': '2-6946', 'Condition_Label': '2-BH7M'}, {'RID': '2-694C', 'Condition_Label': '2-BH7M'}, {'RID': '2-694G', 'Condition_Label': '2-BH7M'}, {'RID': '2-694J', 'Condition_Label': '2-BH7M'}, {'RID': '2-694M', 'Condition_Label': '2

In [42]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

2024-06-17 19:43:01,100 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-06-17 19:43:01,101 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-17 19:43:01,102 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-17 19:43:01,118 - INFO - Checking for updated configuration...
2024-06-17 19:43:01,209 - INFO - Updated configuration found.
2024-06-17 19:43:01,211 - INFO - Attempting to acquire a dependent lock in [/data/lizhiwei/EyeAI_working/Execution_Metadata]
2024-06-17 19:43:01,212 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/Execution_Metadata]...
2024-06-17 19:43:01,216 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_tem